# Deep Learning

Las redes convolucionales son especialmente adecuadas para procesar imágenes. "Aprenden" los coficientes de los filtros que son más adecuados para resolver cada problema concreto.

En el capítulo anterior hemos comprobado su eficacia en el problema relativamente sencillo de reconocimiento de dígitos manuscritos. 

En este tema vamos a poner en marcha varios modelos ya preentrenados que resuelven problemas realistas y se han considerado como avances muy relevantes en visión artificial. (Aparte de esto, OpenCV tiene un [módulo de deep learning](https://docs.opencv.org/4.x/d2/d58/tutorial_table_of_content_dnn.html) para trabajar con modelos de pytorch, tensorflow y otros entornos.)

(Si tienes una GPU con CUDA puedes comparar la diferencia en el tiempo de cómputo respecto al proceso en CPU.)

## Tareas típicas

Algunos problemas típicos de la visión artificial que podemos atacar mediante *deep learning* son los siguientes. (Entre paréntesis se indican las subcarpetas de `code/DL` que contienen ejemplos de código relacionados. Los que tienen prefijo `mp_` están proporcionados por la colección de mediapipe.)

-  Clasificación de la imagen completa (`inception`, `mp_classifier`)
-  Detección de diferentes clases de objetos en la imagen, obteniendo sus *bounding boxes* y categorías (`yolo`, `mp_object`)
-  Extracción de máscaras de segmentación de forma general (`mp_segmenter`, `mp_segmenter_point`, `SAM`, `UNET`)
-  Detección de pose humana con *landmarks* (`mp_face`, `mp_hands`, `mp_humanpose`)
-  *Universal features* preentrenadas que sirven como entrada para clasificadores ligeros (`mp_embedder`, `facerec`, `DINO`)

## Imagenet 

La base de datos [imagenet](http://www.image-net.org/) es un conjunto masivo de imágenes naturales pertenecientes a miles de categorías que se utilizó como benchmark en la competición para conseguir máquinas de vision artificial con capacidad próxima a la humana. Aquí hay un subconjunto con [imágenes reducidas](https://www.kaggle.com/c/tiny-imagenet).)

El paquete keras incluye varias [máquinas preentrenadas](https://keras.io/applications) para resolver este problema. Como se observa en la tabla disponible en esa página, la precisión "top 1" no es espectacular (~78%), pero hay que tener en cuenta que el número de clases es enorme.

El ejemplo de código [inception0.py](../code/DL/inception/inception0.py) muestra la forma de utilizar estas máquinas sobre la imagen en vivo. Se puede elegir entre varios modelos cambiando una variable global y es fácil añadir más. El que parece funcionar mejor de los que he probado es `inception_v3`. Se clasifica la región central marcada con un cuadrado, el resto de la imagen no se tiene en cuenta.

![platanos](../images/demos/platanos.png)

(Esta carpeta incluye también una variante `inception1.py` que hace una captura asíncrona. El objetivo es poner todo el GUI en un hilo y hacer un bucle de procesamiento aparte. Si varios hilos actualizan las ventanas el GUI de opencv se bloquea.)

Al operar sobre imágenes del mundo real, estos modelos cometen algunos errores más o menos justificables, como se muestra en las imágenes de prueba incluidas en la carpeta.

## UNET

En la sesión de prácticas pondremos en marcha un tipo especial de red convolucional que puede utilizarse para obtener máscaras de segmentación. Si el problema es sencillo no requiere muchas imágenes etiquetadas ni excesivo esfuerzo computacional.

## YOLO

Las redes convolucionales anteriores tienen una entrada de tamaño fijo. Para detectar objetos a diferentes escalas y posiciones es necesario ejecutar la máquina sobre múltiples regiones candidatas de la imagen, lo que resulta muy ineficiente.

Un modelo mucho mejor es [YOLO](https://pjreddie.com/darknet/yolo/) (You Only Look Once). En vez de producir como salida un vector de probabilidades de las posibles clases, genera simultáneamente bounding boxes de los posibles objetos con sus probabilidades. Se hace una única ejecución de la red y las detecciones tienen en cuenta todo el contexto de la imagen.

En la carpeta `code/DL/yolo`tenemos un script para utilizar la implementación de YOLO V11 proporcionada por el sistema [ultralytics](https://www.ultralytics.com/).


![yolo](../images/demos/yolo.png)

## Face recognition

En un capítulo anterior hemos visto que el paquete DLIB proporciona un método de detección de caras bastante preciso basado en HOG (histograma de orientaciones de gradiente). Además, incluye un detector de marcadores faciales capaz de estimar la posición de los puntos clave de la cara. Estas posiciones son muy útiles para muchas aplicaciones pero son insuficientes para conseguir un reconocimiento de personas robusto.

Una de las primeras formas de atacar el reconocimiento de caras fue la reducción de dimensión con PCA ([eigenfaces](https://en.wikipedia.org/wiki/Eigenface)). Esta idea puede mejorarse con una transformación no lineal obtenida con una red convolucional profunda con estructura de "cuello de botella". Un avance significativo se produjo con el método [FaceNet](https://www.cv-foundation.org/openaccess/content_cvpr_2015/app/1A_089.pdf), que se basa en **tripletas** de imágenes para construir un vector de 128 features optimizado para conseguir valores iguales con caras de la misma persona y diferentes para personas distintas. Para que esto funcione es necesario alinear muy bien las caras en un marco común, lo que se consigue con los marcadores faciales de DLIB. [Este blog](https://medium.com/@ageitgey/machine-learning-is-fun-part-4-modern-face-recognition-with-deep-learning-c3cffc121d78) tiene una explicación excelente.

El paquete [face recognition](https://github.com/ageitgey/face_recognition) está basado en [esta implementación](https://cmusatyalab.github.io/openface/) de FaceNet. En la carpeta `code/DL/facerec` tenemos un ejemplo de código con nuestro método general de captura con autoStream. Simplemente se compara el descriptor de los modelos con los de las caras detectadas y se clasifica por mínima distancia.

El método funciona muy bien, como se puede comprobar en la prueba siguiente. Hemos cogido de internet las fotos de los componentes de Monty Python:

![models](../images/demos/monty.png)

Veamos si los reconoce con diferentes edades:

![models](../images/demos/monty-rec1.png)

En esta primera prueba, donde los personajes están muy caracterizados, no he conseguido encontrar un modelo de Terry Gilliam que lo reconociera. Pero el modelo joven lo reconoce en la siguiente foto, donde están más mayores:

![models](../images/demos/monty-rec2.png)

Los modelos mayores también reconocen a los personajes jóvenes:

![models](../images/demos/monty-rec3.png)

Pruébalo con la webcam o con tus fotos familiares para hacerte una idea de la precisión del método.

Esta aplicación incluye muchas técnicas importantes utilizadas en visión artificial: un *encoding* (neuronal) obtenido del *alineamiento* (afín --lo estudiaremos la semana siguiente--) de los *landmarks* (basados en casacadas de regresores) de la cara, detectada con *HOG*.

### Alineamiento afín *

A partir de los face landmarks:

![landmarks](../images/demos/face_landmarks.png)

Detectamos 3 posiciones características:
![align1](../images/demos/alignface1.png)

Y transformamos las imágenes a un marco común:

![align2](../images/demos/alignface2.png)

También podemos intercambiarlas:

![fake](../images/demos/fakefaces.png)

## Body landmarks

Otro problema muy complejo en el que el *deep learning* ha supuesto un gran avance es la [detección de la postura humana](https://github.com/CMU-Perceptual-Computing-Lab/openpose). *OpenPose: Real-time multi-person keypoint detection library for body, face, hands, and foot estimation*.

![openpose](../images/demos/openpose.png)


El proyecto [pose animator](https://github.com/yemount/pose-animator/) tiene una demo de un sistema parecido que funciona en el navegador.

El módulo [mediapipe](https://mediapipe.dev/) incluye un detector de pose humana (para una sola persona) y un detector de manos y dedos que funciona bastante bien. Los probaremos en el laboratorio.

## Eliminación de fondo

Máscara de detección del objeto principal en la escena: [rembg](https://github.com/danielgatis/rembg)

## DINO

Recientemente está cobrando importancia el aprendizaje *self-supervised* [DINOV2](https://dinov2.metademolab.com/), con el que se han conseguido *universal features*.

## CLIP

Embedding común de texto e imagen: [Openclip](https://github.com/mlfoundations/open_clip).

## Estimación de profundidad

[Midas](https://github.com/isl-org/MiDaS), [depthpro](https://github.com/apple/ml-depth-pro), etc.

## Enlaces interesantes

Demo interactiva de un perceptrón multicapa: [tensorflow playground](http://playground.tensorflow.org/)

Demo interactiva de una red convolucional: [cnn-explainer](https://poloclub.github.io/cnn-explainer)

Hugging Face demos: [RF-DTR](https://huggingface.co/spaces/SkalskiP/RF-DETR), [VGGT](https://huggingface.co/spaces/facebook/vggt), [InstantMesh](https://huggingface.co/spaces/TencentARC/InstantMesh)

[Dino2](https://dinov2.metademolab.com/)

[Transformer explainer](https://poloclub.github.io/transformer-explainer)

[Papers with code](https://paperswithcode.com/)

[Segment anything](https://segment-anything.com/)

[Masked autoencoders](https://github.com/facebookresearch/mae)

[Keras](https://keras.io/). Su autor, Chollet, tiene un libro excelente sobre deep learning, con sus correspondientes [notebooks](https://github.com/fchollet/deep-learning-with-python-notebooks).

[Geron](https://github.com/ageron/handson-ml3). Autor de otro libro muy bueno, también con notebooks.

[Tensorflow official models](https://github.com/tensorflow/models/tree/master/official)

[Awesome deep learning](https://github.com/ChristosChristofidis/awesome-deep-learning)

[9 fundamental deep learning papers](https://adeshpande3.github.io/adeshpande3.github.io/The-9-Deep-Learning-Papers-You-Need-To-Know-About.html)

[COCO](https://cocodataset.org/#home)

[house numbers dataset](http://ufldl.stanford.edu/housenumbers/)

[plate recognition](https://matthewearl.github.io/2016/05/06/cnn-anpr/) (2016)

[Deep reinforcement learning](https://gym.openai.com/) (Open AI Gym)

[intro a CNN](https://medium.com/@ageitgey/machine-learning-is-fun-part-3-deep-learning-and-convolutional-neural-networks-f40359318721#.alzgfemh0) (2016)

[generative models](https://openai.com/blog/generative-models/) (2016)

[slides cmu](https://www.cs.cmu.edu/~epxing/Class/10715/lectures/DeepArchitectures.pdf) (2017)

[curso Stanford](http://cs231n.stanford.edu/syllabus.html)

[LeCun slides](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.436.894&rep=rep1&type=pdf) (2010?)